In [1]:
##This script is for setting up new tables of headlines
##It should not be run unless you want to overwrite existing data tables!

In [2]:
##Import necessaries
from bs4 import BeautifulSoup
import urllib.request
import sqlite3
import datetime

In [3]:
##Open database
conn = sqlite3.connect('headlines.db') #Change to r'C:\xampp\htdocs\compare-editorial-line\headlines.db' to act on current local database
c = conn.cursor()

In [4]:
##Use to look at data table if needed
#for row in c.execute("SELECT * FROM data"):
   # print(row)

In [5]:
##If need to delete all data from data table
#c.execute('DELETE FROM data') #If need to delete all data

In [6]:
##Create index table
c.execute('''CREATE TABLE idx (
    id,
    newssource)''')

In [7]:
##GUARDIAN
##Get data
r = urllib.request.urlopen('https://www.theguardian.com/uk').read() #get html
soup = BeautifulSoup(r, "lxml") #run html through beautiful soup
#print(soup) #to look at html
today = datetime.date.today() #Store date and time
now = datetime.datetime.now() #time = datetime.datetime.now().time()

#for link in soup.find_all('a'): # this prints all links in document
    #print(link.get('href'))

In [8]:
##Save data
#soup.find_all("a", class_="fc-item__link")
headlines = soup.find_all("span", class_="js-headline-text") #get all headlines on page

# Create table
c.execute('''CREATE TABLE guardian (
    id,
    date,
    time,
    headline,
    url)''')
#Add Guardian to index table
c.execute("INSERT INTO idx VALUES(1, 'guardian')") 

id = 0 #preallocate
for h in headlines:
    #print(h.get_text())
    id += 1
    c.execute("INSERT INTO guardian VALUES(?, ?, ?, ?, '')", (id, today, now, h.get_text())) #store in database


In [9]:
##Repeat for BBC NEWS
##Get data
bbc_r = urllib.request.urlopen('http://www.bbc.co.uk/news/uk').read()
bbc_soup = BeautifulSoup(bbc_r, "lxml")
bbc_today = datetime.date.today()
bbc_now = datetime.datetime.now()
#print(bbc_soup)

In [10]:
##Save data
main = bbc_soup.div(class_="buzzard-item")
medium = bbc_soup.div(class_="pigeon-item__body")
small = bbc_soup.div(class_="pigeon__column pigeon__column--b")
tiny = bbc_soup.div(class_="macaw-item__body")
bbc_divs = main + medium + small + tiny
bbc_headlines = None

# Create table
c.execute('''CREATE TABLE BBCnews (
    id,
    date,
    time,
    headline,
    url)''')
#Add to index table
c.execute("INSERT INTO idx VALUES(2, 'BBCnews')") 

id = 0 #preallocate
for div in bbc_divs:
    headlines = div.find_all("span", class_="title-link__title-text")
    for h in headlines:
        #print(h.get_text())
        id += 1
        c.execute("INSERT INTO BBCnews VALUES(?, ?, ?, ?, '')", (id, bbc_today, bbc_now, h.get_text()))

In [11]:
##Repeat for DAILY MAIL
##Get data
dm_r = urllib.request.urlopen('http://www.dailymail.co.uk/home/index.html').read()
dm_soup = BeautifulSoup(dm_r, "lxml")
dm_today = datetime.date.today()
dm_now = datetime.datetime.now()
#print(dm_soup)

In [12]:
##Save data
headlines = dm_soup.find_all(class_="linkro-darkred")

# Create table
c.execute('''CREATE TABLE dailymail (
    id,
    date,
    time,
    headline,
    url)''')
#Add to index table
c.execute("INSERT INTO idx VALUES(3, 'dailymail')") 

id = 0 #preallocate
for h in headlines:
    htxt = h.get_text()
    htxt = htxt.strip('\n')
    htxt = htxt.strip('\xa0')
    htxt = htxt.replace('\n','')
    id += 1
    #print(htxt)
    c.execute("INSERT INTO dailymail VALUES(?, ?, ?, ?, '')", (id, dm_today, dm_now, htxt))

In [13]:
##Commit to database and close connection
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()